In [1]:
import cv2
print("OpenCV should be 4.8.0.76 Current version:", cv2.__version__)
from typing import List
import numpy as np
import os
import imageio
import cv2
import copy
import glob

OpenCV should be 4.8.0.76 Current version: 4.8.0


In [1]:
import cv2
import numpy as np

# Abrir cámara
cap = cv2.VideoCapture(0)

# ROI (zona de detección)
roi_x, roi_y, roi_w, roi_h = 100, 50, 400, 400

# Patrón esperado (solo número de vértices)
pattern = [4, 5, 6, 3]
sequence = []

# Rango HSV para verde
lower_green = np.array([40, 70, 70])
upper_green = np.array([85, 255, 255])

# Epsilon fijo
epsilon_factor = 0.05

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Dibujar ROI
    cv2.rectangle(frame, (roi_x, roi_y), (roi_x + roi_w, roi_y + roi_h), (255, 0, 0), 2)
    roi = frame[roi_y:roi_y + roi_h, roi_x:roi_x + roi_w]

    # Convertir ROI a HSV y aplicar máscara verde
    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv_roi, lower_green, upper_green)

    # Contornos en la máscara verde
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    roi_area = roi_w * roi_h

    if contours:
        # Seleccionar el contorno más grande
        c = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(c)

        if area > 1000 and area < 0.8 * roi_area:
            epsilon = epsilon_factor * cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, epsilon, True)
            vertices = len(approx)

            # Dibujar contorno ajustado
            approx_shifted = approx + [roi_x, roi_y]
            cv2.drawContours(frame, [approx_shifted], -1, (0, 255, 0), 2)
            cv2.putText(frame, f"Vertices: {vertices}", (roi_x + 10, roi_y + 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Guardar en secuencia al pulsar 'a'
            key = cv2.waitKey(1) & 0xFF
            if key == ord('a'):
                sequence.append(vertices)
                print("Secuencia actual:", sequence)

                # Validación del patrón
                if sequence == pattern:
                    print("Acceso concedido ✅")
                    cv2.putText(frame, "Acceso concedido ✅", (50, 100),
                                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)
                    cv2.imshow("Patron", frame)
                    cv2.waitKey(3000)
                    break  # salir del bucle sin matar el kernel

    cv2.imshow("Patron por vértices", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cierre ordenado
cap.release()
cv2.destroyAllWindows()

Secuencia actual: [4]
Secuencia actual: [4, 5]
Secuencia actual: [4, 5, 6]
Secuencia actual: [4, 5, 6, 3]
Acceso concedido ✅


In [5]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

# Rango HSV para verde
lower_green = np.array([40, 70, 70])
upper_green = np.array([85, 255, 255])

recording = False
trajectory = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_green, upper_green)

    # Detectar círculo verde (contorno más grande en la máscara)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        c = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(c)
        if area > 1000:  # evitar ruido
            M = cv2.moments(c)
            if M["m00"] > 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
                circle_center = (cx, cy)

                # Dibujar puntero verde
                cv2.circle(frame, circle_center, 5, (0, 255, 0), -1)

                # Guardar trayectoria si estamos grabando
                if recording:
                    trajectory.append(circle_center)

    # Dibujar trayectoria acumulada
    for i in range(1, len(trajectory)):
        cv2.line(frame, trajectory[i-1], trajectory[i], (255, 0, 0), 2)

    cv2.putText(frame, "Press 'r' to record, 's' to stop, 'q' to quit",
                (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    cv2.imshow("Circle Tracker", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('r'):
        trajectory = []  # reiniciar trayectoria
        recording = True
        print("Recording started...")
    elif key == ord('s'):
        recording = False
        if len(trajectory) > 10:
            # Evaluar circularidad respecto al centro medio de la trayectoria
            pts = np.array(trajectory)
            cx, cy = np.mean(pts[:,0]), np.mean(pts[:,1])
            dists = np.sqrt((pts[:,0]-cx)**2 + (pts[:,1]-cy)**2)
            mean_r = np.mean(dists)
            std_r = np.std(dists)

            score = max(0, 100 - (std_r/mean_r)*100)
            print(f"Circle score: {score:.2f}/100")

            # Dibujar círculo ajustado
            cv2.circle(frame, (int(cx), int(cy)), int(mean_r), (0, 255, 255), 2)
            cv2.putText(frame, f"Score: {score:.2f}", (50, 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
            cv2.imshow("Circle Tracker", frame)
            cv2.waitKey(3000)
    elif key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Recording started...
Circle score: 82.21/100
Recording started...
Circle score: 80.04/100
Recording started...
Circle score: 51.35/100
Recording started...
Recording started...
Circle score: 32.62/100
Recording started...
Recording started...
Circle score: 80.83/100
Circle score: 80.83/100
Recording started...
Circle score: 28.78/100
Recording started...
Circle score: 83.53/100
Recording started...
Recording started...
Circle score: 79.01/100
Recording started...
Recording started...
Circle score: 82.47/100
